In [2]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt

from utils.roi_measures import mad, ssim
from utils.extraction import extract_patches
from utils.reconstruction import perform_voting

from model3d import Multimodel

Using TensorFlow backend.


In [3]:
curr_patch_shape =  (32, 32, 32)
input_modalities = ['T1']
output_modalities = ['Gen']
output_weights = {'Gen' : 1.0, 'concat' : 0.5}
latent_dim = 32
channels = [2]
to_process = [True, True]
patch_shape = curr_patch_shape
scale = 1

a_model = Multimodel(
    input_modalities, output_modalities, output_weights, latent_dim, channels, patch_shape, to_process, scale)
a_model.build()

Latent dimensions: 32
Fuse latent representations using max
making output: Tensor("enc_T1_act9/add:0", shape=(?, 32, 32, 32, 32), dtype=float32) Tensor("dec_Gen/dec_Gen_act9/add:0", shape=(?, 1, 32, 32, 32), dtype=float32) em_0_dec_Gen
making output: Tensor("enc_T1_act9/add:0", shape=(?, 32, 32, 32, 32), dtype=float32) Tensor("dec_Gen/dec_Gen_act9/add:0", shape=(?, 1, 32, 32, 32), dtype=float32) em_1_dec_Gen
Skipping embedding distance outputs for unimodal model
all outputs:  [u'em_0_dec_Gen/add:0', u'em_1_dec_Gen/add:0']
output dict:  {'em_1_dec_Gen': <function adhoc_loss at 0x7f1e83d3f5f0>, 'em_0_dec_Gen': <function adhoc_loss at 0x7f1e83d3f5f0>}
loss weights:  {'em_1_dec_Gen': 1.0, 'em_0_dec_Gen': 1.0}


In [ ]:
import os
import nibabel as nib

from medical_data import cdr_info, nwbv_info, diff_info

ref_pattern = 'OAS2_0{0:03}_MR{1}'
file_general_pattern = 'OAS2_0{0:03}_MR{1}_{3}_OAS2_0{0:03}_MR{2}'
dataset_location = 'datasets/OASIS/OASIS2/REG/{}/{}.nii.gz'

is_same_to_same = True
step = (16, 16, 16)
threshold = np.int32(0.30 * np.prod(curr_patch_shape[:]))
seg_train = np.empty((0, 1, ) + curr_patch_shape)
ref_train = np.empty((0, 2, ) + curr_patch_shape)
out_train = np.empty((0, 1, ) + curr_patch_shape)
for i in range(1, 100) :
    print '{} :'.format(i),
    
    for j in range(1, 5) :
        diff = 0
        for k in range(j+1, 5) :
            ref_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, j))
            ref_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, j) + '_seg')
            mov_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, k) + '_reg')
            mov_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, k) + '_reg' + '_seg')
            
            if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
                continue
            
            diff = diff + diff_info['OAS2_0{:03}_MR{}'.format(i, k)]
            
            if is_same_to_same and diff > 0.1 :
                break

            if not is_same_to_same and diff < 0.02:
                continue

            ######################################################################################
            volume_init = nib.load(ref_filename).get_data()
#             volume_init = volume_init / volume_init.max()
            
            mask_patches = extract_patches(volume_init != 0, curr_patch_shape, step)

            useful_patches = np.sum(mask_patches, axis=(1, 2, 3)) > threshold
            
            del mask_patches
            
            ref_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_patches = ref_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(ref_prob_filename).get_data() == 1
            
            ref_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_prob_patches = ref_prob_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            ref_train = np.vstack((np.hstack((ref_patches, ref_prob_patches)), ref_train)).astype('float32') ##
            del ref_patches, ref_prob_patches

            ######################################################################################
            volume_init = nib.load(mov_filename).get_data()
#             volume_init = volume_init / volume_init.max()
            
            mov_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_patches = mov_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(mov_prob_filename).get_data() == 1
            
            mov_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_prob_patches = mov_prob_patches[useful_patches].reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            seg_train = np.vstack((mov_prob_patches, seg_train)).astype('float32') ##
            out_train = np.vstack((mov_patches, out_train)).astype('float32')
            del mov_patches, mov_prob_patches
            ######################################################################################
            print '{}->{}'.format(j, k),
    print

In [4]:
def adhoc_loss(y_true, y_pred) :
    mask = K.cast(K.not_equal(y_true, 0), 'float32')
    m = mae(y_true, y_pred*mask)
    cc = K.mean(categorical_crossentropy(
        S((y_true - train_mean) / train_std),
        S((y_pred - train_mean) / train_std)), axis=(1))
    return m + cc

In [12]:
from keras.callbacks import EarlyStopping, ModelCheckpoint

patience = 3

stopper = EarlyStopping(patience=patience)
checkpointer = ModelCheckpoint('models/ag_o1o2_tv_same_alter.h5', save_best_only=True, save_weights_only=True)

N = len(ref_train)
a_model.model.fit(
    [np.hstack((ref_train[:, 0:1], seg_train))],
    [ref_train[:, 0:1], ref_train[:, 0:1]],
    validation_split=0.3, epochs=40, batch_size=16,
    callbacks=[checkpointer, stopper])

Train on 8168 samples, validate on 3501 samples
Epoch 1/40
8168/8168 [==============================] - 303s 37ms/step - loss: 61.5053 - em_0_dec_Gen_loss: 30.7527 - em_1_dec_Gen_loss: 30.7527 - val_loss: 25.6268 - val_em_0_dec_Gen_loss: 12.8134 - val_em_1_dec_Gen_loss: 12.8134
Epoch 2/40
8168/8168 [==============================] - 296s 36ms/step - loss: 21.8121 - em_0_dec_Gen_loss: 10.9060 - em_1_dec_Gen_loss: 10.9060 - val_loss: 12.0897 - val_em_0_dec_Gen_loss: 6.0449 - val_em_1_dec_Gen_loss: 6.0449
Epoch 3/40
8168/8168 [==============================] - 299s 37ms/step - loss: 22.3654 - em_0_dec_Gen_loss: 11.1827 - em_1_dec_Gen_loss: 11.1827 - val_loss: 27.7068 - val_em_0_dec_Gen_loss: 13.8534 - val_em_1_dec_Gen_loss: 13.8534
Epoch 4/40
8168/8168 [==============================] - 301s 37ms/step - loss: 19.4360 - em_0_dec_Gen_loss: 9.7180 - em_1_dec_Gen_loss: 9.7180 - val_loss: 27.1278 - val_em_0_dec_Gen_loss: 13.5639 - val_em_1_dec_Gen_loss: 13.5639
Epoch 5/40
1520/8168 [====>.....

KeyboardInterrupt: 

In [13]:
a_model.model.load_weights('models/ag_o1o2_tv_same_alter.h5')

In [ ]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

file_general_pattern = 'OAS2_0{0:03}_MR{1}_{3}_OAS2_0{0:03}_MR{2}'
dataset_location = 'datasets/OASIS/OASIS2/REG/{}/{}.nii.gz'

mad_results = {}
ssim_results = {}
curr_patch_shape = (32, 32, 32)
step = (16, 16, 16)
for i in range(100, 190) :
    print '{} :'.format(i),
    for j in range(1, 5) :
        for k in range(j+1, 5) :
            ref_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, j))
            ref_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, j) + '_seg')
            mov_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, k) + '_reg')
            mov_prob_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, k) + '_reg' + '_seg')
            
            if not (os.path.exists(ref_filename) and os.path.exists(mov_filename)) :
                continue

            ######################################################################################
            volume_init = nib.load(ref_filename).get_data()
#             volume_init = volume_init / volume_init.max()
            
            ref_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(ref_prob_filename).get_data() == 1
            
            ref_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            ref_prob_patches = ref_prob_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            ref_test = np.hstack((ref_patches, ref_prob_patches)).astype('float32') ##
            del ref_patches, ref_prob_patches
            ######################################################################################
            volume_init = nib.load(mov_filename).get_data()
            
            mov_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_patches = mov_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            volume_init = nib.load(mov_prob_filename).get_data() == 1
            
            mov_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
            mov_prob_patches = mov_prob_patches.reshape((-1, 1, ) + curr_patch_shape)
            ######################################################################################
            seg_test = mov_prob_patches.astype('float32') ##
            del mov_patches, mov_prob_patches
            ######################################################################################
            print '{}->{}'.format(j, k),
    
            pred = a_model.model.predict(
                [np.hstack((ref_test[:, 0:1], seg_test))], verbose=1)[1]
            pred = pred.reshape((-1, ) + curr_patch_shape)
            
            volume = perform_voting(pred, curr_patch_shape, volume_init.shape, step)
    
#             volume = volume * nib.load(mov_filename).get_data().max()
            volume = np.multiply(nib.load(mov_filename).get_data() != 0, volume)

            volume_data = nib.load(mov_filename)
            
            nib.save(nib.Nifti1Image(volume, volume_data.affine),
                     'tv_results/{}_{}_to_{}.nii.gz'.format(i, j, k))

            mov_seg_filename = dataset_location.format(
                file_general_pattern.format(i, j, k, 'to'),
                ref_pattern.format(i, k) + '_reg' + '_seg')

            res = sitk.ReadImage('tv_results/{}_{}_to_{}.nii.gz'.format(i, j, k))
            
            caster = sitk.CastImageFilter()
            caster.SetOutputPixelType(res.GetPixelID())
            
            orig = caster.Execute(sitk.ReadImage(mov_filename))
            seg = caster.Execute(sitk.ReadImage(mov_seg_filename))
            
            thresholder = sitk.BinaryThresholdImageFilter()
            enhanced_vol = caster.Execute(
                thresholder.Execute(seg, -1, -1, 1, 0))

            pairs = [(1, 3)]
            for (a, b) in pairs :                
                thresholder = sitk.BinaryThresholdImageFilter()
                mask = thresholder.Execute(seg, a, b, 1, 0)

                masker = sitk.MaskImageFilter()
                ref_masked = masker.Execute(orig, mask)
                moving_masked = masker.Execute(res, mask)

                matcher = sitk.HistogramMatchingImageFilter()
                matcher.SetNumberOfHistogramLevels(1024)
                matcher.SetNumberOfMatchPoints(15)
                matcher.SetThresholdAtMeanIntensity(True)
                partial_result = matcher.Execute(moving_masked, ref_masked)

                adder = sitk.AddImageFilter()
                enhanced_vol = adder.Execute(enhanced_vol, partial_result)

            masker = sitk.MaskImageFilter()
            mask = thresholder.Execute(seg, 0, 0, 0, 1)
            enhanced_vol = masker.Execute(enhanced_vol, mask)
            sitk.WriteImage(
                enhanced_vol,
                'tv_results/{}_{}_to_{}_cor.nii.gz'.format(i, j, k))

            volume = nib.load('tv_results/{}_{}_to_{}_cor.nii.gz'.format(i, j, k)).get_data()
#             ref_vol = np.float64(nib.load(ref_filename).get_data())
            act_vol = np.float64(nib.load(mov_filename).get_data())
            mad_movgen = mad(volume, act_vol)
            ssim_movgen = ssim(volume, act_vol)

            mad_results['{} {}->{}'.format(i, j, k)] = mad_movgen
            ssim_results['{} {}->{}'.format(i, j, k)] = ssim_movgen
            print '{}-{}: {} - {}'.format(i, k, mad_movgen, ssim_movgen)

In [15]:
from keras.models import Model
import os
import nibabel as nib
import SimpleITK as sitk

file_general_pattern = 'OAS2_0{0:03}_MR{1}_{3}_OAS2_0{0:03}_MR{2}'
dataset_location = 'datasets/OASIS/OASIS2/REG/{}/{}.nii.gz'

tuples = [(1, 2, 1), (1, 2, 2), (1, 3, 3), (1, 4, 4)]
mad_results = {}
ssim_results = {}
curr_patch_shape = (32, 32, 32)
step = (16, 16, 16)
for i in range(100, 190) :
    print '{} :'.format(i),
    for (j, k, a) in tuples :
        ref_filename = dataset_location.format(
            file_general_pattern.format(i, j, k, 'to'),
            ref_pattern.format(i, a))
        ref_prob_filename = dataset_location.format(
            file_general_pattern.format(i, j, k, 'to'),
            ref_pattern.format(i, a) + ('_reg_seg' if k == a else '_seg'))
        mov_filename = ref_filename
        mov_prob_filename = ref_prob_filename
        
        print ref_filename, ref_prob_filename

        if not os.path.exists(ref_filename):
            continue

        ######################################################################################
        volume_init = nib.load(ref_filename).get_data()
#         volume_init = volume_init / volume_init.max()

        ref_patches = extract_patches(volume_init, curr_patch_shape, step)
        ref_patches = ref_patches.reshape((-1, 1, ) + curr_patch_shape)
        ######################################################################################
        volume_init = nib.load(ref_prob_filename).get_data() == 1

        ref_prob_patches = extract_patches(volume_init, curr_patch_shape, step)
        ref_prob_patches = ref_prob_patches.reshape((-1, 1, ) + curr_patch_shape)
        ######################################################################################
        ref_test = np.hstack((ref_patches, ref_prob_patches)).astype('float32') ##
        seg_test = ref_prob_patches.astype('float32') ##
        del ref_patches, ref_prob_patches
        ######################################################################################
        print '{}->{}'.format(a, a),

        pred = a_model.model.predict(
            [np.hstack((ref_test[:, 0:1], seg_test))], verbose=1)[1]
        pred = pred.reshape((-1, ) + curr_patch_shape)

        volume = perform_voting(pred, curr_patch_shape, volume_init.shape, step)

        volume = np.multiply(nib.load(mov_filename).get_data() != 0, volume)

        volume_data = nib.load(mov_filename)

        nib.save(nib.Nifti1Image(volume, volume_data.affine),
                 'tv_results/{}_{}_to_{}.nii.gz'.format(i, a, a))

        volume = nib.load('tv_results/{}_{}_to_{}.nii.gz'.format(i, a, a)).get_data()
        act_vol = np.float64(nib.load(mov_filename).get_data())
        mad_movgen = mad(volume, act_vol)
        ssim_movgen = ssim(volume, act_vol)

        mad_results['{} {}->{}'.format(i, j, k)] = mad_movgen
        ssim_results['{} {}->{}'.format(i, j, k)] = ssim_movgen
        print '{}-{}: {} - {}'.format(i, k, mad_movgen, ssim_movgen)

 100 : datasets/OASIS/OASIS2/REG/OAS2_0100_MR1_to_OAS2_0100_MR2/OAS2_0100_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0100_MR1_to_OAS2_0100_MR2/OAS2_0100_MR1_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 100-2: 0.00315218340035 - 0.999742692018
datasets/OASIS/OASIS2/REG/OAS2_0100_MR1_to_OAS2_0100_MR2/OAS2_0100_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0100_MR1_to_OAS2_0100_MR2/OAS2_0100_MR2_reg_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 100-2: 0.00746897041728 - 0.999042734418
datasets/OASIS/OASIS2/REG/OAS2_0100_MR1_to_OAS2_0100_MR3/OAS2_0100_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0100_MR1_to_OAS2_0100_MR3/OAS2_0100_MR3_reg_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 100-3: 0.00788137971426 - 0.998943302888
datasets/OASIS/OASIS2/REG/OAS2_0100_MR1_to_OAS2_0100_MR4/OAS2_0100_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0100_MR1_to_OAS2_0100_MR4/OAS2_0100_MR4_reg_seg.nii.gz
101 : datasets/OASIS/OASIS2/REG/OAS2_0101_MR

1575/1575 [==============================] - 5s 3ms/step
 109-2: 0.011914151205 - 0.997653231369
datasets/OASIS/OASIS2/REG/OAS2_0109_MR1_to_OAS2_0109_MR3/OAS2_0109_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0109_MR1_to_OAS2_0109_MR3/OAS2_0109_MR3_reg_seg.nii.gz
datasets/OASIS/OASIS2/REG/OAS2_0109_MR1_to_OAS2_0109_MR4/OAS2_0109_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0109_MR1_to_OAS2_0109_MR4/OAS2_0109_MR4_reg_seg.nii.gz
110 : datasets/OASIS/OASIS2/REG/OAS2_0110_MR1_to_OAS2_0110_MR2/OAS2_0110_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0110_MR1_to_OAS2_0110_MR2/OAS2_0110_MR1_seg.nii.gz
datasets/OASIS/OASIS2/REG/OAS2_0110_MR1_to_OAS2_0110_MR2/OAS2_0110_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0110_MR1_to_OAS2_0110_MR2/OAS2_0110_MR2_reg_seg.nii.gz
datasets/OASIS/OASIS2/REG/OAS2_0110_MR1_to_OAS2_0110_MR3/OAS2_0110_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0110_MR1_to_OAS2_0110_MR3/OAS2_0110_MR3_reg_seg.nii.gz
datasets/OASIS/OASIS2/REG/OAS2_0110_MR1_to_OAS2_0110_MR4/OAS2_0110_MR4.nii.gz dat

1575/1575 [==============================] - 5s 3ms/step
 119-2: 0.00366735372313 - 0.999686034333
datasets/OASIS/OASIS2/REG/OAS2_0119_MR1_to_OAS2_0119_MR3/OAS2_0119_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0119_MR1_to_OAS2_0119_MR3/OAS2_0119_MR3_reg_seg.nii.gz
1575/1575 [==============================] - 5s 3ms/step
 119-3: 0.00757181907393 - 0.999045332604
datasets/OASIS/OASIS2/REG/OAS2_0119_MR1_to_OAS2_0119_MR4/OAS2_0119_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0119_MR1_to_OAS2_0119_MR4/OAS2_0119_MR4_reg_seg.nii.gz
120 : datasets/OASIS/OASIS2/REG/OAS2_0120_MR1_to_OAS2_0120_MR2/OAS2_0120_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0120_MR1_to_OAS2_0120_MR2/OAS2_0120_MR1_seg.nii.gz
1575/1575 [==============================] - 5s 3ms/step
 120-2: 0.000959178613785 - 0.999870233751
datasets/OASIS/OASIS2/REG/OAS2_0120_MR1_to_OAS2_0120_MR2/OAS2_0120_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0120_MR1_to_OAS2_0120_MR2/OAS2_0120_MR2_reg_seg.nii.gz
1575/1575 [==============================] -

1575/1575 [==============================] - 5s 3ms/step
 129-2: 0.00816622335089 - 0.998766384055
datasets/OASIS/OASIS2/REG/OAS2_0129_MR1_to_OAS2_0129_MR2/OAS2_0129_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0129_MR1_to_OAS2_0129_MR2/OAS2_0129_MR2_reg_seg.nii.gz
1575/1575 [==============================] - 5s 3ms/step
 129-2: 0.00268889523385 - 0.999777393306
datasets/OASIS/OASIS2/REG/OAS2_0129_MR1_to_OAS2_0129_MR3/OAS2_0129_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0129_MR1_to_OAS2_0129_MR3/OAS2_0129_MR3_reg_seg.nii.gz
1575/1575 [==============================] - 5s 3ms/step
 129-3: 0.00473880390697 - 0.999525549179
datasets/OASIS/OASIS2/REG/OAS2_0129_MR1_to_OAS2_0129_MR4/OAS2_0129_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0129_MR1_to_OAS2_0129_MR4/OAS2_0129_MR4_reg_seg.nii.gz
130 : datasets/OASIS/OASIS2/REG/OAS2_0130_MR1_to_OAS2_0130_MR2/OAS2_0130_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0130_MR1_to_OAS2_0130_MR2/OAS2_0130_MR1_seg.nii.gz
datasets/OASIS/OASIS2/REG/OAS2_0130_MR1_to_OA

1575/1575 [==============================] - 5s 3ms/step
 139-2: 0.00733111434966 - 0.999080245972
datasets/OASIS/OASIS2/REG/OAS2_0139_MR1_to_OAS2_0139_MR3/OAS2_0139_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0139_MR1_to_OAS2_0139_MR3/OAS2_0139_MR3_reg_seg.nii.gz
datasets/OASIS/OASIS2/REG/OAS2_0139_MR1_to_OAS2_0139_MR4/OAS2_0139_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0139_MR1_to_OAS2_0139_MR4/OAS2_0139_MR4_reg_seg.nii.gz
140 : datasets/OASIS/OASIS2/REG/OAS2_0140_MR1_to_OAS2_0140_MR2/OAS2_0140_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0140_MR1_to_OAS2_0140_MR2/OAS2_0140_MR1_seg.nii.gz
1575/1575 [==============================] - 5s 3ms/step
 140-2: 0.00551259948642 - 0.999397067299
datasets/OASIS/OASIS2/REG/OAS2_0140_MR1_to_OAS2_0140_MR2/OAS2_0140_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0140_MR1_to_OAS2_0140_MR2/OAS2_0140_MR2_reg_seg.nii.gz
1575/1575 [==============================] - 5s 3ms/step
 140-2: 0.0192764337165 - 0.994017334219
datasets/OASIS/OASIS2/REG/OAS2_0140_MR1_to_OAS

1575/1575 [==============================] - 6s 4ms/step
 149-2: 0.00874635058357 - 0.998694692499
datasets/OASIS/OASIS2/REG/OAS2_0149_MR1_to_OAS2_0149_MR2/OAS2_0149_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0149_MR1_to_OAS2_0149_MR2/OAS2_0149_MR2_reg_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 149-2: 0.00294539335885 - 0.99976164768
datasets/OASIS/OASIS2/REG/OAS2_0149_MR1_to_OAS2_0149_MR3/OAS2_0149_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0149_MR1_to_OAS2_0149_MR3/OAS2_0149_MR3_reg_seg.nii.gz
datasets/OASIS/OASIS2/REG/OAS2_0149_MR1_to_OAS2_0149_MR4/OAS2_0149_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0149_MR1_to_OAS2_0149_MR4/OAS2_0149_MR4_reg_seg.nii.gz
150 : datasets/OASIS/OASIS2/REG/OAS2_0150_MR1_to_OAS2_0150_MR2/OAS2_0150_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0150_MR1_to_OAS2_0150_MR2/OAS2_0150_MR1_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 150-2: 0.00593024253829 - 0.999337904981
datasets/OASIS/OASIS2/REG/OAS2_0150_MR1_to_OAS

1575/1575 [==============================] - 6s 4ms/step
 159-2: 0.0037940165403 - 0.999660436434
datasets/OASIS/OASIS2/REG/OAS2_0159_MR1_to_OAS2_0159_MR3/OAS2_0159_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0159_MR1_to_OAS2_0159_MR3/OAS2_0159_MR3_reg_seg.nii.gz
datasets/OASIS/OASIS2/REG/OAS2_0159_MR1_to_OAS2_0159_MR4/OAS2_0159_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0159_MR1_to_OAS2_0159_MR4/OAS2_0159_MR4_reg_seg.nii.gz
160 : datasets/OASIS/OASIS2/REG/OAS2_0160_MR1_to_OAS2_0160_MR2/OAS2_0160_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0160_MR1_to_OAS2_0160_MR2/OAS2_0160_MR1_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 160-2: 0.00466473282699 - 0.999474314948
datasets/OASIS/OASIS2/REG/OAS2_0160_MR1_to_OAS2_0160_MR2/OAS2_0160_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0160_MR1_to_OAS2_0160_MR2/OAS2_0160_MR2_reg_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 160-2: 0.00897750059463 - 0.998638693581
datasets/OASIS/OASIS2/REG/OAS2_0160_MR1_to_OAS

1575/1575 [==============================] - 5s 3ms/step
 171-2: 0.0251646313431 - 0.990669863893
datasets/OASIS/OASIS2/REG/OAS2_0171_MR1_to_OAS2_0171_MR2/OAS2_0171_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0171_MR1_to_OAS2_0171_MR2/OAS2_0171_MR2_reg_seg.nii.gz
1575/1575 [==============================] - 5s 3ms/step
 171-2: 0.00749692173981 - 0.99905692973
datasets/OASIS/OASIS2/REG/OAS2_0171_MR1_to_OAS2_0171_MR3/OAS2_0171_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0171_MR1_to_OAS2_0171_MR3/OAS2_0171_MR3_reg_seg.nii.gz
1575/1575 [==============================] - 6s 3ms/step
 171-3: 0.00336062706412 - 0.999700371014
datasets/OASIS/OASIS2/REG/OAS2_0171_MR1_to_OAS2_0171_MR4/OAS2_0171_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0171_MR1_to_OAS2_0171_MR4/OAS2_0171_MR4_reg_seg.nii.gz
172 : datasets/OASIS/OASIS2/REG/OAS2_0172_MR1_to_OAS2_0172_MR2/OAS2_0172_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0172_MR1_to_OAS2_0172_MR2/OAS2_0172_MR1_seg.nii.gz
1575/1575 [==============================] - 6s

1575/1575 [==============================] - 6s 4ms/step
 181-2: 0.00122367915678 - 0.999834534227
datasets/OASIS/OASIS2/REG/OAS2_0181_MR1_to_OAS2_0181_MR2/OAS2_0181_MR2.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0181_MR1_to_OAS2_0181_MR2/OAS2_0181_MR2_reg_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 181-2: 0.0186399123268 - 0.994733187027
datasets/OASIS/OASIS2/REG/OAS2_0181_MR1_to_OAS2_0181_MR3/OAS2_0181_MR3.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0181_MR1_to_OAS2_0181_MR3/OAS2_0181_MR3_reg_seg.nii.gz
1575/1575 [==============================] - 6s 4ms/step
 181-3: 0.0222770745114 - 0.992555326684
datasets/OASIS/OASIS2/REG/OAS2_0181_MR1_to_OAS2_0181_MR4/OAS2_0181_MR4.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0181_MR1_to_OAS2_0181_MR4/OAS2_0181_MR4_reg_seg.nii.gz
182 : datasets/OASIS/OASIS2/REG/OAS2_0182_MR1_to_OAS2_0182_MR2/OAS2_0182_MR1.nii.gz datasets/OASIS/OASIS2/REG/OAS2_0182_MR1_to_OAS2_0182_MR2/OAS2_0182_MR1_seg.nii.gz
1575/1575 [==============================] - 6s